# Welcome to the ADPT ([`audio-diffusion-pytorch-trainer`](https://github.com/archinetai/audio-diffusion-pytorch-trainer)) notebook beta v0.2!

**In this notebook**:

- Train larger than 1B params model. The results are faster and produces better quality music, generating longer audio tracks in one go – up to 87s of context without inpainting at 48kHz!

Use these models for:
- Unconditional audio sample generation
- Infinite audio generation with inpainting!
- Follow the notebook discussions [here](https://github.com/archinetai/audio-diffusion-pytorch/discussions/categories/notebooks)


In [ ]:
#@title <- View Changelog
skip_for_run_all = True #@param {type: 'boolean'}

if skip_for_run_all == False:
  print(
      '''
  v0.1 Update: Set 11, 2022 - flavioschneider

      - Added Dance Diffusion finetune repo

  v0.2 Update: Oct 29, 2022 - twobob

      - added changelog, experiment editor, multiplatform support, and .env generator
      '''
      )

In [ ]:
#@title Check GPU Status
import shutil
def is_tool(name):
    """Check whether `name` is on PATH and marked as executable."""

    # from whichcraft import which
    from shutil import which

    return which(name) is not None
skip_for_run_all = True #@param {type: 'boolean'}

import subprocess
if skip_for_run_all == False:
  if is_tool('nvidia-smi'):
    simple_nvidia_smi_display = True#@param {type:"boolean"}
    if simple_nvidia_smi_display:
        #!nvidia-smi
        nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
        print(nvidiasmi_output)
    else:
        #!nvidia-smi -i 0 -e 0
        nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
        print(nvidiasmi_output)
        nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
        print(nvidiasmi_ecc_note)
  else:
    print('Your system does not have nvidia-smi available as a command')


In [ ]:
#@title Cleardown options
import os
import time
import datetime
from IPython.display import clear_output
from os.path import exists
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import HBox, Label
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
clear_logs_after_setup = False#@param {type:"boolean"}

In [ ]:
try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    is_kaggle = False
    #@markdown Check to connect your Google Drive
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")
    is_kaggle = True if os.path.exists('/kaggle/working') else False
if is_colab:
    if google_drive is True and save_models_to_google_drive is True:
        drive.mount('/content/drive')
        ai_root = '/content/drive/MyDrive/Colab'
        root_path = f'{ai_root}/respiratory'
    else:
        root_path = '/content'
    true_root = root_path # not sure why the two paths
else:
    root_path = '/kaggle/working/' if is_kaggle else os.getcwd()
    true_root = '/kaggle/working/' if is_kaggle else os.getcwd()

print(f'Using this path as root: {true_root}')

In [ ]:
#@title Setup

repo_dir = "repo"

now = datetime.datetime.now()
def done(clear=False):
    now = datetime.datetime.now()
    if not clear:
        clear_output(wait=False)
    print(f"done at {now.hour}:{now.minute}:{now.second}")

import os
!pip install ipywidgets
#!pip uninstall -y audio-diffusion-pytorch-trainer
os.chdir(true_root)
!rm -rf $repo_dir/
#!git clone --branch NoLog https://github.com/twobob/audio-diffusion-pytorch-trainer
#!git clone https://github.com/archinetai/audio-diffusion-pytorch-trainer
!git clone https://github.com/endofu/audio-diffusion-pytorch-trainer $repo_dir
os.chdir(f'{true_root}/{repo_dir}')
!pip install -r requirements.txt
if clear_logs_after_setup:
  done()
print('endofu/audio-diffusion-pytorch-trainer install completed')

In [ ]:
from prompt_toolkit.layout import layout
#@title Choose An Experiment
import ipywidgets as widgets
from ipywidgets import HBox, Label
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
import time
import pandas as pd
out = widgets.Output()

df = df = pd.DataFrame(columns = ['Dropdown_column', 'Float_column'])

def list_files(your_directory):
  return sorted( filter( lambda x: os.path.isfile(os.path.join(your_directory, x)), os.listdir(your_directory) ) )

expdir=f'{true_root}/{repo_dir}/exp/'
archdir=f'{true_root}/{repo_dir}/exp/archived/'
current_dyamic_dir = expdir # presumes LIVE as default

files = list_files(expdir)
archives =  list_files(archdir)

isYouTube = False

def exp_loader(fully_qualified_filepath):
  global isYouTube
  with open(fully_qualified_filepath, 'r') as doc:
    chomp = doc.read()
    isYouTube = True if any( substring for substring in ['YoutubeDataset','CommonVoiceDataset','LibriSpeechDataset'] if substring in chomp   ) else False
    return chomp

def exp_writer(fully_qualified_filepath, string_text):
  global isYouTube
  with open(fully_qualified_filepath, 'w') as doc:
    isYouTube = True if any( substring for substring in ['YoutubeDataset','CommonVoiceDataset','LibriSpeechDataset'] if substring in string_text   ) else False
    doc.write(string_text)

def get_button_layout():
  return Layout(
    display='inline-flex',
    flex_flow='row',
    justify_content='center',
    padding = '5%',
    align_items = 'flex-start'
)

def dropdown_handler(change):
    global drop_down_input
    drop_down_input = change.new
    if change['type'] == 'change': dependent_drop_down.options=dependent_drop_down_elements[change.new]
    current_dyamic_dir = expdir if drop_down.value=='LiveExperiments' else archdir


def dep_dropdown_handler(change):
    global EXPERIMENT
    current_dyamic_dir = expdir if drop_down.value=='LiveExperiments' else archdir
    editor.value = exp_loader(f'{current_dyamic_dir}{dependent_drop_down.value}')
    EXPERIMENT = dependent_drop_down.value.replace('.yaml','')

def on_edit_button_clicked(b):
    editor.value = exp_loader(f'{current_dyamic_dir}/{dependent_drop_down.value}')
    editor.layout.display = "flex"
    editbutton.layout.display = "none"
    savebutton.layout.display = "flex"

def on_save_button_clicked(b):
    global df
    with out:
      exp_writer(f'{current_dyamic_dir}/{dependent_drop_down.value}', editor.value)
      editor.layout.display = "none"
      savebutton.layout.display = "none"
      editbutton.layout.display = "flex"

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

edit_button_item_layout = get_button_layout()

button_item_layout = get_button_layout()
button_item_layout.display = "none"

#setup the default loaded experiment, hidden in the editor
doctxt= exp_loader(f'{current_dyamic_dir}/{files[0]}')

drop_down_input = 'LiveExperiments'
drop_down = widgets.Dropdown(options=('LiveExperiments', 'ArchivedExperiments'),disabled=True) # Toggle for arch view

dependent_drop_down_elements = {}
dependent_drop_down_elements['LiveExperiments'] = files
dependent_drop_down_elements['ArchivedExperiments'] = archives
dependent_drop_down = widgets.Dropdown(options=(dependent_drop_down_elements['LiveExperiments']))

drop_down.observe(dropdown_handler, names='value')
dependent_drop_down.observe(dep_dropdown_handler, names='value')

savebutton = widgets.Button(description='save experiment', layout=button_item_layout)
savebutton.on_click(on_save_button_clicked)

editbutton = widgets.Button(description='edit experiment', layout = edit_button_item_layout)
editbutton.on_click(on_edit_button_clicked)

editor  = widgets.Textarea(
    value=doctxt,
    placeholder='Edit Here',
    description='Experiment:',
    disabled=False,
    layout=Layout(
    display='none',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%',
    height='800px',
    padding = '1%'
)
)

form_items = [
    Box([Label(value='Experiment Editor'),
         drop_down], layout=form_item_layout),
    Box([Label(value='Choose an experiment to edit then run'),
         dependent_drop_down], layout=form_item_layout),
         editor,
         editbutton,
         savebutton
        ]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='99%',
    padding = '1%'
))
display(form)
display(out)
EXPERIMENT = dependent_drop_down.value.replace('.yaml','')

In [ ]:
TMPENV = f'{true_root}/{repo_dir}/.env.tmp'
LIVEENV = f'{true_root}/{repo_dir}/.env'
!cp $TMPENV $LIVEENV
#@title Setup paths, logins and logging
#@markdown Setup the .env file options
custom_log_path = 'logs'#@param {type: 'string'}
custom_data_path = 'data-respiratory'#@param {type: 'string'}
WANDB_PROJECT = 'femke-respiratory'#@param {type: 'string'}
#@markdown weights and biases username
#@markdown Required if using wandb logger
WANDB_ENTITY = 'endofu'#@param {type: 'string'}
#@markdown weights and biases API key
WANDB_API_KEY = '---'#@param {type: 'string'}
#@markdown huggingface API key
#@markdown Required if using Common Voice dataset
HUGGINGFACE_TOKEN = '---'#@param {type: 'string'}

file = open(LIVEENV, "r")
replacement = ""

# using the for loop
for count, line in enumerate(file):
    line = line.strip()
    if 0 == count:
      line=  changes = f"DIR_LOGS=/{custom_log_path}"
    if 1 == count:
      line= changes = f"DIR_DATA=/{custom_data_path}"

    changes = line.replace("WANDB_PROJECT=wandbprojectname", f"WANDB_PROJECT={WANDB_PROJECT}")
    changes = changes.replace("WANDB_ENTITY=wandbuser", f"WANDB_ENTITY={WANDB_ENTITY}")
    changes = changes.replace("WANDB_API_KEY=wandbapikey", f"WANDB_API_KEY={WANDB_API_KEY}")
    changes = changes.replace("HUGGINGFACE_TOKEN=huggingfacetoken", f"HUGGINGFACE_TOKEN={HUGGINGFACE_TOKEN}")
    replacement = replacement + changes + "\n"
with open(LIVEENV, 'w') as f:
    f.write(replacement)
f.close()
os.chdir(true_root)
if clear_logs_after_setup:
  done()

print('options set')

In [ ]:
%env  HYDRA_FULL_ERROR=1
TRAIN_CMD = f'{true_root}/{repo_dir}/train.py'
EXPERIMENT_CMD = f'exp={EXPERIMENT}'
PATH_CMD = '' if isYouTube else f'++datamodule.dataset.path={true_root}/{custom_data_path}'
CKPT = "2023-09-26-19-58-27"
CKPT_CMD = f'+ckpt={true_root}/{custom_log_path}/ckpts/{CKPT}/last.ckpt' if CKPT else ''


#@title Run the finetune
## n GPU TRAINING with added data dir.

os.chdir(true_root)
print(f'Running the command: !python {true_root}/audio-diffusion-pytorch-trainer/train.py exp={EXPERIMENT} trainer.gpus=1 ++datamodule.dataset.path={true_root}/{custom_data_path}')
!python $TRAIN_CMD $EXPERIMENT_CMD trainer.gpus=1 $PATH_CMD $CKPT_CMD
